In [18]:
!pip install pycaret

     |████████████████████████████████| 301 kB 558 kB/s            
     |████████████████████████████████| 15.6 MB 7.8 MB/s            
     |████████████████████████████████| 114 kB 58.3 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 636 kB 41.9 MB/s            
     |████████████████████████████████| 25.9 MB 4.3 kB/s             
     |████████████████████████████████| 10.4 MB 38.1 MB/s            
     |████████████████████████████████| 167 kB 38.9 MB/s            
     |████████████████████████████████| 24.2 MB 53.8 MB/s            
     |████████████████████████████████| 184 kB 67.7 MB/s            
     |████████████████████████████████| 1.0 MB 40.6 MB/s            
     |████████████████████████████████| 14.8 MB 62.2 MB/s            
     |████████████████████████████████| 58 kB 3.9 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 79 kB 6.2 MB/s             
  Created wheel for 

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.linear_model import Lasso,Ridge
from pycaret.regression import *

In [46]:
data = pd.read_csv('../input/diamonds/diamonds.csv')

In [47]:
data.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [48]:
data.columns

Index(['Unnamed: 0', 'carat', 'cut', 'color', 'clarity', 'depth', 'table',
       'price', 'x', 'y', 'z'],
      dtype='object')

In [49]:
data.drop('Unnamed: 0',axis=1,inplace=True)

In [50]:
data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [51]:
y = data['price']
X = data.drop('price',axis=1)

In [52]:
y

0         326
1         326
2         327
3         334
4         335
         ... 
53935    2757
53936    2757
53937    2757
53938    2757
53939    2757
Name: price, Length: 53940, dtype: int64

In [53]:
pd.DataFrame(X)

,carat,cut,color,clarity,depth,table,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,6.15,6.12,3.74


In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.1+ MB


In [55]:
print(f"Cuts: {len(X['cut'].unique())}")
print(f"Colors: {len(X['color'].unique())}")
print(f"Clarity: {len(X['clarity'].unique())}")

Cuts: 5
Colors: 7
Clarity: 8


In [56]:
encoder = LabelEncoder()

X['cut'] = encoder.fit_transform(X['cut'])
cut_mappings = {index:label for index,label in enumerate(encoder.classes_)}

X['color'] = encoder.fit_transform(X['color'])
color_mappings = {index:label for index,label in enumerate(encoder.classes_)}

X['clarity'] = encoder.fit_transform(X['clarity'])
clarity_mappings = {index:label for index,label in enumerate(encoder.classes_)}


In [57]:
print(cut_mappings)
print(color_mappings)
print(clarity_mappings)

{0: 'Fair', 1: 'Good', 2: 'Ideal', 3: 'Premium', 4: 'Very Good'}
{0: 'D', 1: 'E', 2: 'F', 3: 'G', 4: 'H', 5: 'I', 6: 'J'}
{0: 'I1', 1: 'IF', 2: 'SI1', 3: 'SI2', 4: 'VS1', 5: 'VS2', 6: 'VVS1', 7: 'VVS2'}


In [58]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,0.23,2,1,3,61.5,55.0,3.95,3.98,2.43
1,0.21,3,1,2,59.8,61.0,3.89,3.84,2.31
2,0.23,1,1,4,56.9,65.0,4.05,4.07,2.31
3,0.29,3,5,5,62.4,58.0,4.20,4.23,2.63
4,0.31,1,6,3,63.3,58.0,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...
53935,0.72,2,0,2,60.8,57.0,5.75,5.76,3.50
53936,0.72,1,0,2,63.1,55.0,5.69,5.75,3.61
53937,0.70,4,0,2,62.8,60.0,5.66,5.68,3.56
53938,0.86,3,4,3,61.0,58.0,6.15,6.12,3.74


In [59]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [60]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8
0,0.006237,0.50,0.166667,0.428571,0.513889,0.230769,0.367784,0.067572,0.076415
1,0.002079,0.75,0.166667,0.285714,0.466667,0.346154,0.362197,0.065195,0.072642
2,0.006237,0.25,0.166667,0.571429,0.386111,0.423077,0.377095,0.069100,0.072642
3,0.018711,0.75,0.833333,0.714286,0.538889,0.288462,0.391061,0.071817,0.082704
4,0.022869,0.25,1.000000,0.428571,0.563889,0.288462,0.404097,0.073854,0.086478
...,...,...,...,...,...,...,...,...,...
53935,0.108108,0.50,0.000000,0.285714,0.494444,0.269231,0.535382,0.097793,0.110063
53936,0.108108,0.25,0.000000,0.285714,0.558333,0.230769,0.529795,0.097623,0.113522
53937,0.103950,1.00,0.000000,0.285714,0.550000,0.326923,0.527002,0.096435,0.111950
53938,0.137214,0.75,0.666667,0.428571,0.500000,0.288462,0.572626,0.103905,0.117610


In [61]:
s = setup(data,target='price')

,Description,Value
0,session_id,994
1,Target,price
2,Original Data,"(53940, 10)"
3,Missing Values,False
4,Numeric Features,6
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(37757, 26)"


In [62]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,282.9534,291873.0056,539.7045,0.9816,0.1057,0.0783,5.2430
lightgbm,Light Gradient Boosting Machine,290.1058,302751.3968,549.7855,0.9809,0.1113,0.0851,0.2740
et,Extra Trees Regressor,273.1260,302439.4177,549.5999,0.9809,0.0941,0.0678,6.4250
rf,Random Forest Regressor,276.9656,309742.0154,555.8508,0.9805,0.0933,0.0672,8.2400
xgboost,Extreme Gradient Boosting,288.2537,313160.4837,559.1764,0.9803,0.1012,0.0752,26.5830
gbr,Gradient Boosting Regressor,407.3228,549968.5603,740.9360,0.9654,0.2177,0.1302,1.9960
dt,Decision Tree Regressor,365.7489,566044.9148,751.7525,0.9643,0.1271,0.0877,0.1470
knn,K Neighbors Regressor,528.3726,972986.0976,985.9822,0.9387,0.1911,0.1473,0.4530
lasso,Lasso Regression,742.0811,1287539.1396,1134.3742,0.9188,0.5808,0.3873,0.0700
ridge,Ridge Regression,743.9004,1292554.7996,1136.5872,0.9185,0.5780,0.3914,0.0180


In [63]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,shuffle=True)

In [66]:
std_model = LinearRegression()
l1_model = Lasso(alpha=1)
l2_model = Ridge(alpha=1)

std_model.fit(X_train,y_train)
l1_model.fit(X_train,y_train)
l2_model.fit(X_train,y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [67]:
print(f"----Without Regulazrization: {std_model.score(X_train,y_train)}")
print(f"----Lasso Regulazrization: {l1_model.score(X_train,y_train)}")
print(f"----Ridge Regulazrization: {l2_model.score(X_train,y_train)}")


----Without Regulazrization: 0.8849142196438767
----Lasso Regulazrization: 0.8841099302407621
----Ridge Regulazrization: 0.8846004266082039


In [75]:
l2_model = Ridge(alpha=0.1)
l2_model.fit(X_train,y_train)
print(f"----Ridge Regulazrization: {l2_model.score(X_train,y_train)}")

----Ridge Regulazrization: 0.8849033451018313
